# <center>Spellcheck with a Hash Table</center>


# Code

In [63]:
import pandas as pd
import numpy as np
import re
from Levenshtein import distance

## Hash Table Function

In [98]:
class HashTable:
    def __init__(self, size=100):
        # Initialize an empty list to represent the hash table with a specified size
        self.size = size
        self.word_table = [None] * size

    def hash_function(self, word):
        # Simple hash function: sum of ASCII values of characters in the word
        return sum(ord(char) for char in word) % self.size

    def insert(self, word, frequency):
        # Hash the word to get the index
        index = self.hash_function(word)

        # If the index is empty, create a list to store collisions
        if self.word_table[index] is None:
            self.word_table[index] = []

        # Append the word and its frequency to the list at the index
        self.word_table[index].append((word, frequency))

    def get(self, word):
        # Hash the word to get the index
        index = self.hash_function(word)

        # If the index is not empty, search for the word in the list at the index
        if self.word_table[index] is not None:
            for stored_word, stored_freq in self.word_table[index]:
                if stored_word == word:
                    return stored_freq

        # Word not found
        return None

    def find_similar_words(self, word):
        # Find words with Levenshtein distance of 1 from the given word, ordered by frequency
        similar_words = []

        # Iterate through each index in the word_table
        for stored_words in self.word_table:
            if stored_words is not None:
                # Iterate through each stored word and frequency in the list at the current index
                for stored_word, stored_freq in stored_words:
                    # Check if the Levenshtein distance between the given word and stored_word is 1
                    if distance(word, stored_word) == 1:
                        similar_words.append((stored_word, stored_freq))

        # Sort similar_words by frequency in descending order
        similar_words.sort(key=lambda x: x[1], reverse=True)

        # Extract the words for the final result
        similar_words = [similar_word for similar_word, _ in similar_words]

        return similar_words

#### Time Complexity

- **Init:** O(1)

- **Hash Function:** O(k), where k is the length of the input word

- **Insert:** O(1) on average, but in the worst case (with many collisions), it can approach O(n), where n is the number of elements in the collision list.

- **Get:** O(1) on average, but in the worst case (with many collisions), it can approach O(n), where n is the number of elements in the collision list.

- **Find Similar Words:** O(N * L^2), where N is the number of words in the dataset, and L is the max length of a word


#### Space Complexity

- **Init:** O(n), where n is the specified size of the hash table

- **Hash Function:** O(1)

- **Insert:** O(1)

- **Get:** O(1)

- **Find Similar Words:** O(M), where M is the number of similar words found

## Making the Hash Tables

In [100]:
from wordfreq import word_frequency, top_n_list

# Set the language to English
language = 'en'

# Get a list of all the words in the dataset
word_list = top_n_list(language, 1000000)

# Create a df
df = pd.DataFrame(word_list, columns=['Words'])

# Add a new column 'Frequency' with the frequency of each word
df['Frequency'] = df['Words'].apply(lambda word: word_frequency(word, language))

In [101]:
# Create a hash table
hash_table = HashTable()

# Populate the hash table with words and their frequencies
for _, row in df.iterrows():
    hash_table.insert(row['Words'], row['Frequency'])

## Spell-Check

Sample Sentence: ew the grosses thing just happende to me i just wann go to the stor and buty a new car

Sample Paragraph: The quick brow fox jumps ovr the lazy doog. The sun is shining bright in the bueatiful sky, and the birds are chirping happly. However, somthing seems amis, as the wind whispers a mystereous tale through the rustling leaves. It's a peculiar moment, full of odditiies and unxpected surprises.

In [105]:
from colorama import Fore, Style

# Prompt the user to write a sentence
user_sentence = input("Enter a sentence: ")

# Use re.sub to replace most punctuation with an empty string
user_sentence = re.sub(r'[.,!?/()\[\]{}":;]', ' ', user_sentence)

# Split the sentence into words and convert to lowercase
words = user_sentence.lower().split()

# Store formatted information for each word
formatted_words = []

# Display information for each word
for word in words:
    frequency = hash_table.get(word)

    if frequency is not None: #if word in dataset
        if frequency < 3.63e-08: #if word in below 50th percentile of frequency
            formatted_word = f"{Fore.LIGHTRED_EX}{word}{Style.RESET_ALL}"
        elif frequency < 2.19e-07: #if word in 75th percentile of frequency
            formatted_word = f"{Fore.YELLOW}{word}{Style.RESET_ALL}"
        elif frequency >= 0.000003: #if word in 95th percentile of frequency
            formatted_word = f"{Fore.GREEN}{word}{Style.RESET_ALL}"
        else: #if word in 75th percentile of frequency
            formatted_word = f"{Fore.BLACK}{word}{Style.RESET_ALL}"
        formatted_words.append(formatted_word)
    else: #if word not in dataset
        formatted_word = f"{Fore.RED}{word}{Style.RESET_ALL}"
        formatted_words.append(formatted_word)

# Print the sentence with colored words
colored_sentence = ' '.join(formatted_words)
print(f"\n\033[1mCorrected Sentence:\033[0m {colored_sentence}")

#Print the frequencies of the word
print("\n\033[1mWord\t\tFrequencies\033[0m")
print("\n")
for word in words:
    frequency = hash_table.get(word)
    if frequency is not None:
        if frequency < 2.19e-07:
            similar_start_words = hash_table.find_similar_words(word)
            if similar_start_words:
                print(f"{word}\t\t{frequency:.10e}\t\t Did you mean one of these words: {similar_start_words}")
            else:
                print(f"{word}\t\t{frequency:.10e}\t\t Alternate word could not be determined")
        else:
            print(f"{word}\t\t{frequency:.10e}")
    else:
        similar_start_words = hash_table.find_similar_words(word)
        if similar_start_words:
            print(f"{word}\t\tN/A\t\t Did you mean one of these words: {similar_start_words}")
        else:
            print(f"{word}\t\tN/A\t\t Alternate word could not be determined")
    print("\n")

#Print a key for what the colors mean
print(f"{Fore.BLACK}\n\n\033[1mColor Code Key:\033[0m")
print(f"{Fore.RED}Red{Style.RESET_ALL} - Word not found - spelling error")
print(f"{Fore.LIGHTRED_EX}Dark Red{Style.RESET_ALL} - Word extremely rare, likely spelling mistake")
print(f"{Fore.YELLOW}Yellow{Style.RESET_ALL} - Very rare word, check if correct")
print(f"{Fore.BLACK}Black{Style.RESET_ALL} - Fairly common word")
print(f"{Fore.GREEN}Green{Style.RESET_ALL} - Very common word")

Enter a sentence:  ew the grosses thing just happende to me i just wann go to the stor and buty a new car



Corrected Sentence: ew the grosses thing just happende to me i just wann go to the stor and buty a new car

Word		Frequencies


ew		3.8000000000e-06


the		5.3700000000e-02


grosses		3.7200000000e-07


thing		5.2500000000e-04


just		2.6900000000e-03


happende		N/A		 Did you mean one of these words: ['happend', 'happends', 'happended', 'happene']


to		2.6900000000e-02


me		3.0200000000e-03


i		1.2300000000e-02


just		2.6900000000e-03


wann		1.2600000000e-07		 Did you mean one of these words: ['want', 'wanna', 'ann', 'warn', 'wang', 'wan', 'mann', 'wand', 'wynn', 'swann', 'winn', 'wank', 'wane', 'wasn', 'vann', 'dann', 'wani', 'wenn', 'cann', 'yann', 'kann', 'wain', 'hann', 'wana', 'rann', 'jann', 'gann', 'bann', 'tann', 'wans', 'fann', 'wanne', 'pann', 'lann', 'oann', 'sann', 'wano', 'wanz', 'zann', 'wnn', 'wahn', 'nann', 'waen', 'waun', 'waan', 'wany', 'wayn', 'wanni', 'waln', 'iann']


go		1.0700000000e-03


to		2.6900000000e-02


the		5.3700000000e-02


stor		2.2400000000e-0

## Determining Colors for Words

This section was used to determine the thresholds for different colors 

In [77]:
import numpy as np

# Calculate percentiles
percentiles = np.percentile(df['Frequency'], np.arange(0, 101, 5))

# Print the words at different percentiles along with corresponding frequency
for p, (percentile, frequency) in zip(np.arange(0, 101, 5), zip(percentiles, np.percentile(df['Frequency'], np.arange(0, 101, 5)))):
    word_at_percentile = df[df['Frequency'] == frequency]['Words'].values
    print(f"{p}th percentile - Frequency: {frequency}: {word_at_percentile}")


0th percentile - Frequency: 1.02e-08: ['1includes' '1no' '1usd' ... '💷' '🕓' '🤞🏽']
5th percentile - Frequency: 1.12e-08: ['1re' '2ar' '2fl' ... '🔃' '🔬' '🚟']
10th percentile - Frequency: 1.23e-08: ['0pt' '1em' '1jz' ... '📆' '🔠' '🔻']
15th percentile - Frequency: 1.35e-08: ['0st' '0x000' '1at' ... '🔄' '🙏🏿' '🤦🏾\u200d♀️']
20th percentile - Frequency: 1.51e-08: ['1on' '1ss' '2da' ... '👉🏼' '👊🏽' '🦐']
25th percentile - Frequency: 1.7e-08: ['0z' '2be' '2bed' ... '🐑' '👢' '🕐']
30th percentile - Frequency: 1.95e-08: ['1ad' '1to' '2rs' ... '💺' '🗽' '🚪']
35th percentile - Frequency: 2.24e-08: ['0ut' '1mph' '2ms' ... '👊🏼' '💒' '🚑']
40th percentile - Frequency: 2.57e-08: ['0i' '0mm' '1ba' ... '😾' '🚓' '🚔']
45th percentile - Frequency: 3.02e-08: ['0ver' '1dx' '1tbsp' ... '📝' '😼' '🤕']
50th percentile - Frequency: 3.63e-08: ['0nly' '1pp' '2face' ... '💬' '🤦🏽\u200d♂️' '🧠']
55th percentile - Frequency: 4.47e-08: ['0l' '1v2' '2h00' ... '🔔' '🤝' '🤷🏾\u200d♂️']
60th percentile - Frequency: 5.62e-08: ['1i' '1year' '1z

# Documentation

### Findings

- Finding a good dataset is incredibly difficult. Nearly all publicly available datasets of the english language contain words that aren't actually words, as seen in the one chosen above. As such, some leeway must be given for determining what is or isnt a word.
    - Good datasets must exist somewhere, as spellcheck does work elsewhere. In those cases, the datasets also likely contain word combinations or phrases, so as to check for grammar or for correct usage of a word. 

- Populating the hash table takes the most time of any of the functions. Once it has been populated, actually doing the spell check is relatively fast. Even if a paragraph is inserted, the code runs relatively quickly
    - This would make implementing the code in multiple languages relatively easy, so long as the user was prompted to select a language before typing. However, the hash table for every language option should be loaded beforehand, otherwise it will cause the spellcheck to run much slower
    
- Although this approach is far more rudimentary, the basic principles of checking against a dataset of existing words is how spell check operates

### Performance Analysis

- Apart from populating the hash table, the code opperates fairly quickly and smoothly. 
    - The speed of populating the hash table is somewhat unavoidable (regardless of the size of the initialized HashTable), as the dataset is simply too large. Even simply putting the data into a df is slow
- The code takes longer when a word is spelled incorrectly as it is then required to parse through the hash table to see if it can find a better replacement word.